# Data Science Task

In [ ]:
import pandas as pd
import numpy as np
df = pd.read_excel('Sample_Longlist_Data.xlsx')

In [ ]:
#Idea about Different columns, Null-count and data type
df.info()

In [ ]:
#size of data
df.shape

In [ ]:
#initial preview
df.head()

In [ ]:
#All the column names
df.columns

In [ ]:
#let us see unique values for categorical data
df['category'].value_counts() # 1839 implies all are of 1 category ie. Garden
df['category'].unique()
df['sellerlink'].unique() # 1839 unique sellers
# df['sellerlink-url'] and df['sellerstorefront-url'] contains info about seller hence can be dropped
df['sellerratings'].value_counts() #requires further inquiry
#sample brand names and URL's can be dropped until further required
df['Date Added'].value_counts() #1839 and all are 1 single date

In [ ]:
df1 = df.drop(['sellerlink','sellerlink-url','sellerstorefront-url','Sample Brand URL','Date Added','category','Sample brand name'],axis = 1)

In [ ]:
df1

In [ ]:
#Removing 'Business name' in seller business name
df1['seller business name'] = df1['seller business name'].str.replace('^Business Name:','',regex=True)
df1

In [ ]:
df1['sellerproductcount'] = df1['sellerproductcount'].fillna('0').str.replace(',', '').astype(int)

In [ ]:
df1

In [ ]:
#Business address with null is removed
df1 = df1.dropna(subset=['businessaddress'])
df1

In [ ]:

#Let us look at Business address and parse on basis of Countries
def map_country(code):
    if code[-2:] == 'CN':
        return 'China'
    elif code[-2:] == 'DE':
        return 'Germany'
    elif code[-2:] == 'US':
        return 'USA'
    else:
        return 'Unknown'

In [ ]:
df1['Seller Country'] = df1['businessaddress'].apply(map_country)

In [ ]:
df1

In [ ]:
#dropping china
df1 = df1[df1["Seller Country"].str.contains("China") == False]

In [ ]:
df1

In [ ]:
#To separate email and phone numbers
import re
pattern = r'(?:E-Mail|Email|email):\s*([^\s]+(?:\.com|\.de|\.at))'
df1['Email'] = df1['sellerdetails'].str.extract(pattern, flags=re.IGNORECASE)


In [ ]:
df1

In [ ]:
pattern = r'(?:Tel\.|Tel|Telefon):\s*([\d-]+)'
df1['Telephone'] = df1['sellerdetails'].str.extract(pattern, flags=re.IGNORECASE)

In [ ]:
df1

In [ ]:
#Positive Ratings
df1['% positive rating'] = df1['sellerratings'].str.extract(r'(\d+)\s*%')


In [ ]:
df1['% positive rating'] = df1['% positive rating'].fillna('0').astype(int)

In [ ]:
df1['% positive rating'].dtype

In [ ]:
df1

In [ ]:
#df2 = df1.drop(columns=['% positive rateing'],axis=1)

In [ ]:
df2 = df1
file_name = 'Project_DS.xlsx'
  
# saving the excel
df2.to_excel(file_name)

In [ ]:
#total ratings
df2['Total Ratings'] = df1['sellerratings'].str.extract(r'\((\d+)\s*ratings\)')

In [ ]:
df2
file_name='Project_DS'
df2.to_excel('Project_DS')

# Analysis Time :)

In [ ]:
df2.head(50)

In [ ]:
# Finding best sellers in each country
df2['Seller Country'].unique()
df2['Seller Country'].value_counts()

In [ ]:
# Finding best sellers in Germany, USA and unknown
df_germany = df2[df2['Seller Country'] == 'Germany']

In [ ]:
df_germany

In [ ]:
df2['% positive rating'].dtype

In [ ]:
df_g= df_germany.sort_values("% positive rating", axis = 0, ascending = False, inplace = False)

In [ ]:
df_g = df_g.sort_values("sellerproductcount", axis = 0, ascending = False, inplace = False)

In [ ]:
df_g

In [ ]:
df_usa = df2[df2['Seller Country'] == 'USA']
df_u = df_usa.sort_values("% positive rating", axis = 0, ascending = False, inplace = False)
df_u = df_usa.sort_values("sellerproductcount", axis = 0, ascending = False, inplace = False)

In [ ]:
df_u

In [ ]:
df_uk = df2[df2['Seller Country'] == 'Unknown']
df_u1 = df_uk.sort_values("% positive rating", axis = 0, ascending = False, inplace = False)
df_u1 = df_uk.sort_values("sellerproductcount", axis = 0, ascending = False, inplace = False)
df_u1

In [ ]:
# Finding best selllers on basis of Hero product rating
df2['HeroProdSum'] = df['Hero Product 1 #ratings'] + df['Hero Product 2 #ratings']
sorted_hp = df2.sort_values(by='HeroProdSum', ascending=False)

In [ ]:
sorted_hp

In [ ]:
# worst sellers on basis of negative ratings
df_worst = df2.sort_values("Max % of negative seller ratings - last 90 days", axis = 0, ascending = False, inplace = False)

In [ ]:
df_worst

In [ ]:
#mask = (~df2['Email'].isna()) | (~df2['Telephone'].isna())

In [ ]:
# Sort the DataFrame based on the mask and columns in descending order
#df2_contact = df2[mask].sort_values(by=['Email', 'Telephone'], ascending=True).append(df2[~mask])

In [ ]:
# sellers ranking on base of ease of contact
# Create boolean masks for different cases
both_available_mask = ~df2['Email'].isna() & ~df2['Telephone'].isna()
either_available_mask = ~both_available_mask & (~df2['Email'].isna() | ~df2['Telephone'].isna())
both_nan_mask = df2['Email'].isna() & df2['Telephone'].isna()

# Sort the DataFrame based on the masks and columns
sorted_df_contact = (
    df2[both_available_mask].sort_values(by=['Email', 'Telephone'])
    .append(df2[either_available_mask].sort_values(by=['Email', 'Telephone']))
    .append(df2[both_nan_mask])
)


In [ ]:
sorted_df_contact